In [4]:
import json
import os

class User:
    def __init__(self, username, trial_attributes=None, files=None):
        self.username = username
        self.trial_attributes = trial_attributes or {}  # Dictionary to store trial attributes
        self.files = files or {}  # Dictionary to store file paths and status

    def add_trial_attribute(self, trial_name, attributes):
        """Adds trial attributes to the user's trial_attributes dictionary."""
        self.trial_attributes[trial_name] = attributes

    def add_file(self, filename, status):
        """Adds a file and its status to the user's files dictionary."""
        self.files[filename] = status

    def to_json(self, filepath):
        """Saves the user object to a JSON file."""
        data = {
            "username": self.username,
            "trial_attributes": self.trial_attributes,
            "files": self.files
        }
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=4)

    @classmethod
    def from_json(cls, filepath):
        """Loads a user object from a JSON file."""
        with open(filepath, 'r') as f:
            data = json.load(f)
        return cls(data["username"], data["trial_attributes"], data["files"])

    @classmethod
    def load_from_directory(cls, directory):
        """Loads all files from a directory and creates a user object."""
        username = os.path.basename(directory)  # Assumes directory name is the username
        user = cls(username)

        for filename in os.listdir(directory):
            filepath = os.path.join(directory, filename)
            if os.path.isfile(filepath):
                # You can add logic here to determine the file status (e.g., 'U' or None)
                # For now, let's assume all files are 'U'
                user.add_file(filename, 'U')

        return user

# Example Usage (based on the image):

# 1. Create a User object from the directory structure
current_directory = os.getcwd()
user_directory = os.path.join(current_directory, "users")
user = User.load_from_directory(user_directory)

# 2. Add trial attributes (if you have them)
user.add_trial_attribute("side_step_left_1", {"some_attribute": "value"})
user.add_trial_attribute("side_step_right_1", {"another_attribute": 123})
user.add_trial_attribute("sprint_1", {"speed": "fast"})

# 3. Save the User object to a JSON file
settings_file = os.path.join(current_directory, "settings.json")
user.to_json(settings_file)

# 4. Load the User object from the JSON file
loaded_user = User.from_json(settings_file)

# 5. Access the user's attributes and files
print(loaded_user.username)
print(loaded_user.trial_attributes)
print(loaded_user.files)


users
{'side_step_left_1': {'some_attribute': 'value'}, 'side_step_right_1': {'another_attribute': 123}, 'sprint_1': {'speed': 'fast'}}
{}


# Create classes and functions

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import c3d
import xml.etree.ElementTree as ET
import opensim as osim
import json
import msk_modelling_python as msk

class File:
    def __init__(self, path):
        self.path = path
        self.name = os.path.basename(path)
        self.extension = os.path.splitext(path)[1]
        
        if not os.path.isfile(path):
            print(f"\033[93mFile not found: {path}\033[0m")
            return        
        
        try:
            endheader_line = msk.classes.osimSetup.find_file_endheader_line(path)
        except:
            print(f"Error finding endheader line for file: {path}")
            endheader_line = 0
        # Read file based on extension
        try:
            if self.extension == '.csv':
                self.data = msk.pd.read_csv(path)
            elif self.extension == '.json':
                self.data = msk.bops.import_json_file(path)
            elif self.extension == '.xml':
                self.data = msk.bops.XMLTools.load(path)
            else:
                try:
                    self.data = msk.pd.read_csv(path, sep="\t", skiprows=endheader_line)
                except:
                    self.data = None
                    
            # add time range for the data
            try:
                self.time_range = [self.data['time'].iloc[0], self.data['time'].iloc[-1]]
                try:
                    self.time_range = [self.data['Time'].iloc[0], self.data['Time'].iloc[-1]]
                except:
                    pass
            except:
                self.time_range = None
        
        except Exception as e:
            print(f"Error reading file: {path}")
            print(e)
            self.data = None
            self.time_range = None
            
class Trial:
    '''
    Class to store trial information and file paths, and export files to OpenSim format
    
    Inputs: trial_path (str) - path to the trial folder
    
    Attributes:
    path (str) - path to the trial folder
    name (str) - name of the trial folder
    og_c3d (str) - path to the original c3d file
    c3d (str) - path to the c3d file in the trial folder
    markers (str) - path to the marker trc file
    grf (str) - path to the ground reaction force mot file
    ...
    
    Methods: use dir(Trial) to see all methods
    
    '''
    def __init__(self, trial_path=None):  
        
        if not trial_path:
            trial_path = msk.ui.select_folder('Select trial folder')
                  
        self.path = trial_path
        self.name = os.path.basename(self.path)
        self.subject = os.path.basename(os.path.dirname(self.path))
        self.c3d = os.path.join(os.path.dirname(self.path), self.name + '.c3d')
        self.markers = File(os.path.join(self.path,'markers_experimental.trc'))
        self.grf = File(os.path.join(self.path,'Visual3d_SIMM_grf.mot'))
        self.emg_csv = os.path.join(self.path,'processed_emg_signals.csv')
        self.emg = File(os.path.join(self.path,'processed_emg_signals.mot'))
        self.ik = File(os.path.join(self.path,'Visual3d_SIMM_input.mot'))
        self.id = File(os.path.join(self.path,'inverse_dynamics.sto'))
        self.so_force = File(os.path.join(self.path,'Results_SO_and_MA', f'{self.subject}_StaticOptimization_force.sto'))
        self.so_activation = File(os.path.join(self.path, 'Results_SO_and_MA', f'{self.subject}_StaticOptimization_activation.sto'))
        self.jra = File(os.path.join(self.path,'joint_reacton_loads.sto'))
        
        # load muscle analysis files
        self.ma_targets = ['_MomentArm_', '_Length.sto']
        self.ma_files = []
        try:
            files = os.listdir(os.path.join(self.path, 'Results_SO_and_MA'))
            for file in files:
                if file.__contains__(self.ma_targets[0]) or file.__contains__(self.ma_targets[1]):
                    self.ma_files.append(File(os.path.join(self.path, 'Results_SO_and_MA', file)))
        except:
            self.ma_files = None
                    
        # settings files
        self.grf_xml = File(os.path.join(self.path,'GRF_Setup.xml'))
        self.actuators_so = File(os.path.join(self.path,'actuators_SO.xml'))
        
        self.settings_json = File(os.path.join(self.path,'settings.json'))
                             
    def check_files(self):
        '''
        Output: True if all files exist, False if any file is missing
        '''
        files = self.__dict__.values()
        all_files_exist = True
        for file in files:
            try:
                if not os.path.isfile(file):
                    print('File not found: ' + file)
                    all_files_exist = False
            except:
                pass
        return all_files_exist
    
    def header_mot(self,df,name):

            num_rows = len(df)
            num_cols = len(df.columns) 
            inital_time = df['Time'].iloc[0]
            final_time = df['Time'].iloc[-1]
            df_range = f'{inital_time}  {final_time}'


            return f'name {name}\n datacolumns {num_cols}\n datarows {num_rows}\n range {df_range} \n endheader'
        
    def csv_to_mot(self):
        
        emg_data = msk.bops.pd.read_csv(self.emg_csv)

        fs = int(1/(emg_data['time'][1] - emg_data['time'][0]))

        time = emg_data['time']

        # start time from new time point
        start_time = time.iloc[0]
        end_time = time.iloc[-1] - time.iloc[0] + start_time

        num_samples = len(emg_data)
        #num_samples = int((end_time - start_time) / (1/fs))
        new_time = np.linspace(start_time, end_time, num_samples)

        emg_data['time'] = new_time

        # Define a new file path 
        new_file_path = os.path.join(self.emg_csv.replace('.csv', '.mot'))

        # Save the modified DataFrame
        emg_data.to_csv(new_file_path, index=False)  # index=False prevents adding an extra index column

        # save to mot
        header = self.header_mot(emg_data, "processed_emg_signals")

        mot_path = new_file_path.replace('.csv','.mot')
        with open(mot_path, 'w') as f:
            f.write(header + '\n')  
            # print column names 
            f.write('\t'.join(map(str, emg_data.columns)) + '\n')
            for index, row in emg_data.iterrows():
                f.write('\t'.join(map(str, row.values)) + '\n')  
        
        print(f"File saved: {mot_path}")

    def create_settings_json(self, overwrite=False):
        if os.path.isfile(self.settings_json) and not overwrite:
            print('settings.json already exists')
            return
        
        settings_dict = self.__dict__
        msk.bops.save_json_file(settings_dict, self.settings_json)
        print('trial settings.json created in ' + self.path)
    
    def exportC3D(self):
        msk.bops.c3d_osim_export(self.og_c3d) 

    def change_grf_xml_path(self):

        try:
            self.tree = ET.parse(self.grf_xml.path)
            self.root = self.tree.getroot()
            self.root.find('.//datafile').text = self.grf.path
            
            self.tree.write(self.grf_xml.path)
            
            print(f"GRF file path updated in {self.grf_xml.path}")
        except Exception as e:
            print(f"Error loading XML file: {e}")
            return None

    def save_json_file(self, data=None, jsonFilePath=None):
        if not jsonFilePath:
            jsonFilePath = self.settings_json
        
        if not data:
            data = self
            
        data = data.__dict__

        with open(jsonFilePath, 'w') as f:
            json.dump(data, f, indent=4)

        json_data = msk.bops.import_json_file(jsonFilePath)
        return json_data
    
    def to_json(self):
        msk.bops.save_json_file(self.__dict__, jsonFilePath = self.settings_json)
        print('settings.json created in ' + self.settings_json)
    
    def run_IK(osim_modelPath, trc_file, resultsDir):
        '''
        Function to run Inverse Kinematics using the OpenSim API.
        
        Inputs:
                osim_modelPath(str): path to the OpenSim model file
                trc_file(str): path to the TRC file
                resultsDir(str): path to the directory where the results will be saved
        '''

        # Load the TRC file
        import pdb; pdb.set_trace()
        tuple_data = import_trc_file(trc_file)
        df = pd.DataFrame.from_records(tuple_data, columns=[x[0] for x in tuple_data])
        column_names = [x[0] for x in tuple_data]
        if len(set(column_names)) != len(column_names):
            print("Error: Duplicate column names found.")
        # Load the model
        osimModel = osim.Model(osim_modelPath)                              
        state = osimModel.initSystem()

        # Define the time range for the analysis
        
        initialTime = TRCData.getIndependentColumn()
        finalTime = TRCData.getLastTime()

        # Create the inverse kinematics tool
        ikTool = osim.InverseKinematicsTool()
        ikTool.setModel(osimModel)
        ikTool.setStartTime(initialTime)
        ikTool.setEndTime(finalTime)
        ikTool.setMarkerDataFileName(trc_file)
        ikTool.setResultsDir(resultsDir)
        ikTool.set_accuracy(1e-6)
        ikTool.setOutputMotionFileName(os.path.join(resultsDir, "ik.mot"))

        # print setup
        ikTool.printToXML(os.path.join(resultsDir, "ik_setup.xml"))         

        # Run inverse kinematics
        print("running ik...")                                             
        ikTool.run()

    def run_inverse_kinematics(model_file, marker_file, output_motion_file):
        # Load model and create an InverseKinematicsTool
        model = osim.Model(model_file)
        ik_tool = osim.InverseKinematicsTool()

        # Set the model for the InverseKinematicsTool
        ik_tool.setModel(model)

        # Set the marker data file for the InverseKinematicsTool
        ik_tool.setMarkerDataFileName(marker_file)

        # Specify output motion file
        ik_tool.setOutputMotionFileName(output_motion_file)

        # Save setup file
        ik_tool.printToXML('setup_ik.xml')

        # Run Inverse Kinematics
        ik_tool.run()

    def run_ID(self, osim_modelPath, coordinates_file, external_loads_file, output_file, LowpassCutoffFrequency=6, run_tool=True):
        
        try: 
            model = osim.Model(osim_modelPath)
        except Exception as e:
            print(f"Error loading model: {osim_modelPath}")
            print(e)
            return
        
        results_folder = os.path.dirname(output_file)
        
        # Setup for excluding muscles from ID
        exclude = osim.ArrayStr()
        exclude.append("Muscles")
        # Setup for setting time range
        IKData = osim.Storage(coordinates_file)

        # Create inverse dynamics tool, set parameters and run
        id_tool = osim.InverseDynamicsTool()
        id_tool.setModel(model)
        id_tool.setCoordinatesFileName(coordinates_file)
        id_tool.setExternalLoadsFileName(external_loads_file)
        id_tool.setOutputGenForceFileName(output_file)
        id_tool.setLowpassCutoffFrequency(LowpassCutoffFrequency)
        id_tool.setStartTime(IKData.getFirstTime())
        id_tool.setEndTime(IKData.getLastTime())
        id_tool.setExcludedForces(exclude)
        id_tool.setResultsDir(results_folder)
        id_tool.printToXML(os.path.join(results_folder, "setup_ID.xml"))
        
        if run_tool:
            id_tool.run()
    
    def export_analog(self, c3dFilePath=None):
        if not c3dFilePath:
            print('C3D file path not provided')
            return
        
        reader = c3d.Reader(open(c3dFilePath, 'rb'))

        # get analog labels, trimmed and replace '.' with '_'
        analog_labels = reader.analog_labels
        analog_labels = [label.strip() for label in analog_labels]
        analog_labels = [label.replace('.', '_') for label in analog_labels]

        # get analog labels, trimmed and replace '.' with '_'
        fs = reader.analog_rate

        # add time to dataframe
        first_frame = reader.first_frame / fs
        final_time = (reader.first_frame + reader.frame_count-1) / fs
        time = msk.np.arange(first_frame / fs, final_time, 1 / fs)  
        num_frames = len(time)
        df = msk.pd.DataFrame(index=range(num_frames),columns=analog_labels)
        df['time'] = time

        # move time to first column
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        df = df[cols] 

        # loop through frames and add analog data to dataframe
        for i_frame, points, analog in reader.read_frames():
            
            # get row number and print loading bar
            i_row = i_frame - reader.first_frame
            # msk.ut.print_loading_bar(i_row/num_frames)
            
            # convert analog data to list
            analog_list  = analog.data.tolist()
            
            # loop through analog channels and add to dataframe
            for i_channel in range(len(analog_list)):
                channel_name = analog_labels[i_channel]
                
                # add channel to dataframe
                df.loc[i_row, channel_name] = analog[i_channel][0]
                
        # save emg data to csv
        df.to_csv(self.emg_csv)
        
        # save to mot
        self.csv_to_mot()
    
class openSim:
    def __init__(self, leg = 'l', subjects =['PC002','PC003','PC006', 'PC013', 'TD006', 'TD013', 'TD017', 'TD021', 'TD023', 'TD026'], trials_to_load = ['trial1','trial2','trial3','normal1', 'normal2', 'normal3', 'crouch1', 'crouch2', 'crouch3'], trial_number = 1):
        try:
            self.code_path = os.path.dirname(__file__)
        except:
            self.code_path = os.getcwd()
        
        self.simulations_path = os.path.join(os.path.dirname(self.code_path), 'Simulations')
        self.subjects = {}
        
        for subject in subjects:
            self.subjects[subject] = {}
            self.subjects[subject]['model'] = os.path.join(self.simulations_path, subject, subject + '_scaled.osim')
            
            for trial in trials_to_load:                
                self.trial_path = os.path.join(self.simulations_path, subject, f'{trial}_{leg}{trial_number}')
                try:
                    trial = Trial(self.trial_path)
                    self.subjects[subject][trial.name] = trial 
                except Exception as e:
                    self.subjects[subject][trial] =  None
                    # print(f"Error loading trial: {self.trial_path}")
                    # print(e)
        

        self.ik_columns = ["hip_flexion_" + leg, "hip_adduction_" + leg, "hip_rotation_" + leg, "knee_angle_" + leg, "ankle_angle_" + leg]
        self.id_columns = ["hip_flexion_" + leg + "_moment", "hip_adduction_" + leg + "_moment", "hip_rotation_" + leg + "_moment", "knee_angle_" + leg + "_moment", "ankle_angle_" + leg + "_moment"]
        self.force_columns = ["add_long_" + leg, "rect_fem_" + leg, "med_gas_" + leg, "semiten_" + leg,"tib_ant_" + leg]


        self.titles = ["Hip Flexion", "Hip Adduction", "Hip Rotation", "Knee Flexion", "Ankle Plantarflexion"]
        self.titles_muscles = ["Adductor Longus", "Rectus Femoris", "Medial Gastrocnemius", "Semitendinosus", "Tibialis Anterior"]

    # Time Normalisation Function 
    def time_normalised_df(self, df, fs=None):
        if not isinstance(df, msk.pd.DataFrame):
            raise Exception('Input must be a pandas DataFrame')
        
        if not fs:
            try:
                fs = 1 / (df['time'][1] - df['time'][0])  # Ensure correct time column
            except KeyError:
                raise Exception('Input DataFrame must contain a column named "time"')
            
        normalised_df = msk.pd.DataFrame(columns=df.columns)

        for column in df.columns:
            if column == 'time':  # Skip time column
                continue	
            normalised_df[column] = msk.np.zeros(101)

            currentData = df[column].dropna()  # Remove NaN values

            timeTrial = msk.np.linspace(0, len(currentData) / fs, len(currentData))  # Original time points
            Tnorm = msk.np.linspace(0, timeTrial[-1], 101)  # Normalize to 101 points

            normalised_df[column] = msk.np.interp(Tnorm, timeTrial, currentData)  # Interpolate

        return normalised_df

    def plot_single_trial(self, show = False):
        #Read .mot files
        with open(self.mot_file, "r") as file:
            lines = file.readlines()

        # Find the line where actual data starts (usually after 'endheader')
        for i, line in enumerate(lines):
            if "endheader" in line:
                start_row = i + 1  # Data starts after this line
                break
        else:
            start_row = 0  # If 'endheader' is not found, assume no header

        # Load data using Pandas
        self.df_ik = msk.pd.read_csv(self.mot_file, delim_whitespace=True, start_row=start_row)
        self.df_id = msk.pd.read_csv(self.id_file, sep="\t", start_row=6)
        self.df_force = msk.pd.read_csv(self.force_file, sep="\t", start_row=14)

        # Apply normalisation to both IK (angles) and ID (moments) data
        self.df_ik_normalized = self.time_normalised_df(df=self.df_ik)
        self.df_id_normalized = self.time_normalised_df(df=self.df_id)
        self.df_force_normalized = self.time_normalised_df(df=self.df_force)

        # Ensure time is normalized to 101 points
        time_normalized = msk.np.linspace(0, 100, 101)  
 
        # select the specified columns         
        self.ik_data = self.df_ik_normalized[self.ik_columns]
        self.id_data = self.df_id_normalized[self.id_columns]
        self.force_data = self.df_force_normalized[self.force_columns]
            
        # Define the layout 
        fig, axes = plt.subplots(2, 5, figsize=(15, 4)) 

        #Plot IK (angles)
        for i, col in enumerate(self.ik_columns):
            ax = axes[0,i]
            ax.plot(time_normalized, self.ik_data[col], color='red')  # Main curve
            ax.set_title(self.titles[i])
            if i == 0:
                ax.set_ylabel("Angle (deg)")
            ax.grid(True)

        #Plot ID (moments)
        for i, col in enumerate(self.id_columns):
            ax = axes[1,i]
            ax.plot(time_normalized, self.id_data[col], color='blue')  # Main curve
            ax.set_title(self.titles[i])
            if i == 0:
                ax.set_ylabel("Moment (Nm)")
            ax.set_xlabel("% Gait Cycle")
            ax.grid(True)

        plt.tight_layout()


        # PLOT MUSCLE FORCES 
        fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(15, 4), sharex=True)

        for i, col in enumerate(self.force_columns):
            ax = axes[i]
            ax.plot(time_normalized, self.force_data[col], color='green')
            ax.set_title(self.titles_muscles[i])
            if i == 0:
                ax.set_ylabel("Force (N)")
            ax.set_xlabel("% Gait Cycle")
            ax.grid(True)

        plt.tight_layout()
        
        if show:
            plt.show()

    def plot_multiple_trials(self, show=False):
        self.df_ik_list = []  # Store loaded DataFrames
        
        for subject in self.subjects:
            for trial in self.subjects[subject]:
                trial_obj = self.subjects[subject][trial]
                if trial_obj:
                    self.df_ik_list.append(trial_obj.ik.data)
                    
        for file in self.mot_files:  # Loop through each file
            with open(file, "r") as f:
                lines = f.readlines()

            # Load data using Pandas
            df = msk.pd.read_csv(file, delim_whitespace=True, skiprows=5)
            self.df_ik_list.append(df)

        # Normalize all loaded IK data
        self.df_ik_normalized_list = []  # Store normalized DataFrames

        for df in self.df_ik_list:  # Loop through each loaded DataFrame
            df_normalized = self.time_normalised_df(df=df)  # Apply normalization
            self.df_ik_normalized_list.append(df_normalized)  # Store normalized DataFrame

        # Ensure time is normalized to 101 points
        time_normalized = msk.np.linspace(0, 100, 101)

        # Select the specified columns from normalized data
        self.ik_data_list = []  # Store DataFrames with only the required columns

        for df_normalized in self.df_ik_normalized_list:  # Loop through each normalized DataFrame
            if set(self.ik_columns).issubset(df_normalized.columns):  # Check if columns exist
                self.ik_data_list.append(df_normalized[self.ik_columns])  # Select only specified columns
            else:
                print("Warning: Some specified columns are missing in a file.")

        # Plot mean and sd
        # Check if IK data exists
        if not self.ik_data_list:
            print("No IK data available to plot!")
        else:
            # Convert list of DataFrames to a single NumPy array
            combined_df = np.array([df.values for df in self.ik_data_list])  # Shape: (num_trials, num_timepoints, num_columns)

            # Check if data is properly structured
            if combined_df.shape[0] < 2:
                print("Not enough trials to calculate mean and standard deviation!")
            else:
                # Compute Mean and Standard Deviation
                mean_values = np.mean(combined_df, axis=0)
                std_values = np.std(combined_df, axis=0)

                # Normalize time from 0 to 100% Gait Cycle
                time_values = np.linspace(0, 100, combined_df.shape[1])

                # Create a shared figure for all subplots
                fig, axes = plt.subplots(nrows=1, ncols=len(self.ik_columns), figsize=(20, 5), sharex=True)

                if len(self.ik_columns) == 1:
                    axes = [axes]  # If only one column, ensure it's iterable

                for i, col in enumerate(self.ik_columns):
                    ax = axes[i]

                    # Plot mean line
                    ax.plot(time_values, mean_values[:, i], color='red', label="Mean", linewidth=2)

                    # Shade the standard deviation range
                    ax.fill_between(time_values, mean_values[:, i] - std_values[:, i],
                                    mean_values[:, i] + std_values[:, i], color='red', alpha=0.2, label="SD Range")

                    # Formatting
                    ax.set_title(col)
                    ax.set_xlabel("Gait Cycle (%)")
                    ax.set_xlim(0, 100)  # X-axis from 0% to 100% of the gait cycle
                    ax.grid(True)

                    # Set Y-label only for the first subplot
                    if i == 0:
                        ax.set_ylabel("Angle (Degrees)")
                        ax.legend()


                plt.tight_layout()

                if show:
                    plt.show()

def export_c3d(c3dFilePath):
    analog_file_path = os.path.join(os.path.dirname(c3dFilePath),'analog.csv')
    
    # if the file already exists, return the file
    if os.path.isfile(analog_file_path):
        df = msk.pd.read_csv(analog_file_path)
        return df
    
    print('Exporting analog data to csv ...')
    
    # read c3d file
    reader = c3d.Reader(open(c3dFilePath, 'rb'))

    # get analog labels, trimmed and replace '.' with '_'
    analog_labels = reader.analog_labels
    analog_labels = [label.strip() for label in analog_labels]
    analog_labels = [label.replace('.', '_') for label in analog_labels]

    # get analog labels, trimmed and replace '.' with '_'
    first_frame = reader.first_frame
    num_frames = reader.frame_count
    fs = reader.analog_rate

    # add time to dataframe
    initial_time = first_frame / fs
    final_time = (first_frame + num_frames-1) / fs
    time = np.arange(first_frame / fs, final_time, 1 / fs) 

    df = msk.pd.DataFrame(index=range(num_frames),columns=analog_labels)
    df['time'] = time
    
    # move time to first column
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]    
    
    # loop through frames and add analog data to dataframe
    for i_frame, points, analog in reader.read_frames():
        
        # get row number and print loading bar
        i_row = i_frame - reader.first_frame
        # msk.ut.print_loading_bar(i_row/num_frames)
        
        # convert analog data to list
        analog_list  = analog.data.tolist()
        
        # loop through analog channels and add to dataframe
        for i_channel in range(len(analog_list)):
            channel_name = analog_labels[i_channel]
            
            # add channel to dataframe
            df.loc[i_row, channel_name] = analog[i_channel][0]
    
    # save emg data to csv   
    df.to_csv(analog_file_path)
    print('analog.csv exported to ' + analog_file_path)  
    
    return df

## load trial

In [2]:
trial_path = msk.ui.select_folder('Select trial folder')
trial = msk.classes.Trial(trial_path=trial_path)

file does not exist
file extension does not match any of the bops options
file extension does not match any of the bops options
file extension does not match any of the bops options
file extension does not match any of the bops options
file extension does not match any of the bops options
file extension does not match any of the bops options


In [3]:
trial.files

[None, None, None, None, None, None, None, None, None, None, None, None, None]